In [1]:

import pandas as pd
import numpy as np

In [ ]:
# PROCESO DE LIMPIEZA DE LOS DATAFRAME:

In [ ]:
# IMPORTO CSV DE EPDATA-> EVOLUCION DEL AGUA DISPONIBLE EN ESPAÑA
df1 = pd.read_csv ('Data/evolucion_del_agua_disponible_en_españa.csv', sep= ';', thousands= '.' )
df1.shape()

In [ ]:
# COMPRUEBO LA INFORMACION
df1.info()

In [ ]:
#ELIMINO LAS FILAS DEL CSV QUE NO TIENEN DATOS

df1_1 = df1.drop(df1.index[17:],axis= 0)
df1_1.drop ('Periodo',axis=1, inplace= True)

In [ ]:
# CONVIERTO LA COLUMNA AÑO EN FECHA
df1_1['Año'] = pd.to_datetime(df1_1['Año'])

In [ ]:
# DOY FORMATO INT A LAS DOS COLUMNAS
df1_1['No potabilizada'].astype(int)
df1_1['Para su potabilización'].astype(int)
    

In [ ]:
# NORAMLIZO LOS TADOS PARA PODER TENER LAS MISMAS METRICAS QUE EN EL DATAFRAME DEL CONSUMO DE AGUA POR TIPO DE INDUSTRIA

df1_1['Para su potabilización'] = df1_1['Para su potabilización'].apply(lambda x: ((x*10**3)/10**8))
df1_1['Para su potabilización']

In [ ]:
# NORAMLIZO LOS TADOS PARA PODER TENER LAS MISMAS METRICAS QUE EN EL DATAFRAME DEL CONSUMO DE AGUA POR TIPO DE INDUSTRIA

df1_1['No potabilizada'] = df1_1['No potabilizada'].apply(lambda x: ((x*10**3)/10**8))


In [ ]:
#RENOMBRO LAS COLUMNAS CON LA METRICA QUE ESTOY UTILIZANDO
df1_1 = df1_1.rename(columns= {'No potabilizada' : 'No potabilizada Valor 10^9m3/año', 'Para su potabilización' : 'Para su potabilización 10^9m3/año'})

In [ ]:
# CALCULO EL TOTAL DEL AGUA POR AÑO Y LO AÑADO A LA COLUMNA QUE CREO 
df1_1['Total Agua']= df1_1['No potabilizada Valor 10^9m3/año']+ df1_1['Para su potabilización 10^9m3/año']


In [ ]:
#GUARDO EL CSV NORMALIZADO
df1_1.to_csv('Data/evolucion_del_agua_disponible_en_españa_normalizado.csv', sep= ';', index= False)

In [ ]:
#IMPORTO EL GRAFICO DE FAO -> CONSUMO DE AGUA POR SECTOR
df2 = pd.read_csv ('Data/Extraccion de agua total por sector en Espana.csv', sep= ',', index_col= False)


In [ ]:
#VEO LA INFORMACION QUE HAY
df2.head(50)

In [ ]:
#COMPRUEBO LOS TIPOS
df2.info()

In [ ]:
#ELIMINO LAS FILAS DEL CSV QUE NO TIENEN DATOS Y LOS AÑOS QUE NO ME INTERESAN, ADEMÁS DE ELIMNINAR LAS COLUMNAS QUE NO ME INTERESA Y RENOMBRAR LAS QUE QUIERO RENOMBRAR
df2_1 = df2.drop(df2.index[0:1],axis= 0)
df2_1 = df2_1.drop(df2_1.index[27:],axis= 0)
df2_1 = df2_1.drop(df2_1.index[6:7],axis= 0)
df2_1 = df2_1.drop(df2_1.index[12:13],axis= 0)
df2_1 = df2_1.drop(df2_1.index[18:19],axis= 0)
df2_1.reset_index(inplace=True, drop=True)# despues de eliminar la primera fila, reseteo el index y dejamos la misma columna inicial del indice

df2_1.drop (['Area','Area Id','Variable Id','Symbol','Md'],axis=1, inplace= True)
df2_1 = df2_1.rename(columns= {'Variable Name' :'Uso', 'Year' : 'Año', 'Value' : 'Valor 10^9m3/año' })

In [ ]:
#PASO A INT LA COLUMNA AÑO
df2_1['Año'] = df2_1['Año'].astype(int)

In [ ]:
# COMO LA INFORMACION APARECE DE 5 EN 5 AÑOS, TENGO QUE AÑADIR LOS AÑOS QUE NO APARECEN
# EL CALCULO ESTÁ BASADO EN LA DIFERENCIA ENTRE UN AÑO, EL ANTERIOR Y LA DIFERENCIA DIVIDIDA ENTRE 5, PARA CADA SECTOR:

In [ ]:
#ESTO YA FUNCION PARA EXTRACCION DE AGUA AGRICOLA PARA TODOS LOS AÑOS
filtro = df2_1['Uso'].isin(['Extracción de agua agrícola'])
extrac_agri_df = df2_1[filtro]


# con esto extraigo los valores de los años, desde el principio hasta el final
lista_datos_valor_df = [] # esta lista va a contener los valores de la columna valor del dataframe
for a in list(range(len(extrac_agri_df))):
    lista_datos_valor_df.append(extrac_agri_df.iloc[a][2])
    
#print(lista_datos_valor_df)

lista_valores_a_sumar = [] # esta lista va a contener el valor que hay que sumar a cada año nuevo que se añade

for a in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_a_sumar.append((lista_datos_valor_df[a+1]-lista_datos_valor_df[a])/5)

lista_a_añadir_df = []# esta va a ser la lista final, con los valores a añadir por cada año a añadir

for valordf in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_años_nuevos = []# lista intermedia que guarda los valore por cada vuelta de grupo nuevo de años 
    lista_valores_años_nuevos.append(extrac_agri_df.iloc[valordf][2])
        
    for a in list(range(5)):   
        
        lista_valores_años_nuevos.append(lista_valores_años_nuevos[a]+lista_valores_a_sumar[valordf]) 
        valor_año_nuevo = lista_valores_años_nuevos[a]+lista_valores_a_sumar[a]

    lista_valores_años_nuevos.pop(0)
    lista_valores_años_nuevos.pop(len(lista_valores_años_nuevos)-1)
    lista_a_añadir_df.append(lista_valores_años_nuevos)

lista_a_añadir_df.pop(len(lista_a_añadir_df)-1)

 
 #CONVIERTO EL ARRAY DE DOS DIMENSIONES DE LOS AÑOS NUEVOS A AÑADIR-> lista_a_añadir_df EN UNA LISTA 
lista_nueva_valor = []
for indice in list(range(len(lista_a_añadir_df))):
    for subindice in list(range(len(lista_a_añadir_df[indice]))):
        lista_nueva_valor.append(lista_a_añadir_df[indice][subindice])
        
#AÑADO LA LISTA DE LOS AÑOS Y LA NUEVA LISTA OBTENIDA DEL ARRAY DE DOS DIMENSIONES EN EL DATAFRAM DF2_1


lista_años_nuevos = [1998,1999,2000,2001,2003,2004,2005,2006,2008,2009,2010,2011,2013,2014,2015,2016]

for indice in list(range(len(lista_nueva_valor))):
    df2_1.loc[len(df2_1)] =  ['Extracción de agua agrícola', lista_años_nuevos[indice], lista_nueva_valor[indice]]

In [ ]:
#ESTO YA FUNCION PARA EXTRACCION DE AGUA INSTURIAL PARA TODOS LOS AÑOS
filtro = df2_1['Uso'].isin(['Extracción de agua industrial'])
extrac_agri_df = df2_1[filtro]

# con esto extraigo los valores de los años, desde el principio hasta el final
lista_datos_valor_df = [] # esta lista va a contener los valores de la columna valor del dataframe
for a in list(range(len(extrac_agri_df))):
    lista_datos_valor_df.append(extrac_agri_df.iloc[a][2])
    

lista_valores_a_sumar = [] # esta lista va a contener el valor que hay que sumar a cada año nuevo que se añade

for a in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_a_sumar.append((lista_datos_valor_df[a+1]-lista_datos_valor_df[a])/5)

lista_a_añadir_df = []# esta va a ser la lista final, con los valores a añadir por cada año a añadir

for valordf in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_años_nuevos = []# lista intermedia que guarda los valore por cada vuelta de grupo nuevo de años 
    lista_valores_años_nuevos.append(extrac_agri_df.iloc[valordf][2])
        
    for a in list(range(5)):   
        # print ('printeamos vuelta del bucle for (a): ', a) 
        
        lista_valores_años_nuevos.append(lista_valores_años_nuevos[a]+lista_valores_a_sumar[valordf]) 
        valor_año_nuevo = lista_valores_años_nuevos[a]+lista_valores_a_sumar[a]

    lista_valores_años_nuevos.pop(0)
    lista_valores_años_nuevos.pop(len(lista_valores_años_nuevos)-1)
    lista_a_añadir_df.append(lista_valores_años_nuevos)

lista_a_añadir_df.pop(len(lista_a_añadir_df)-1)

    
 #CONVIERTO EL ARRAY DE DOS DIMENSIONES DE LOS AÑOS NUEVOS A AÑADIR-> lista_a_añadir_df EN UNA LISTA 
lista_nueva_valor = []
for indice in list(range(len(lista_a_añadir_df))):
    for subindice in list(range(len(lista_a_añadir_df[indice]))):
        lista_nueva_valor.append(lista_a_añadir_df[indice][subindice])
        
#AÑADO LA LISTA DE LOS AÑOS Y LA NUEVA LISTA OBTENIDA DEL ARRAY DE DOS DIMENSIONES EN EL DATAFRAM DF2_1


lista_años_nuevos = [1998,1999,2000,2001,2003,2004,2005,2006,2008,2009,2010,2011,2013,2014,2015,2016]

for indice in list(range(len(lista_nueva_valor))):
    df2_1.loc[len(df2_1)] =  ['Extracción de agua industrial', lista_años_nuevos[indice], lista_nueva_valor[indice]]   

In [ ]:
#ESTO YA FUNCION PARA EXTRACCION DE AGUA MUNICIPAL PARA TODOS LOS AÑOS
filtro = df2_1['Uso'].isin(['Extracción de agua municipal'])
extrac_agri_df = df2_1[filtro]

# con esto extraigo los valores de los años, desde el principio hasta el final
lista_datos_valor_df = [] # esta lista va a contener los valores de la columna valor del dataframe
for a in list(range(len(extrac_agri_df))):
    lista_datos_valor_df.append(extrac_agri_df.iloc[a][2])
    

lista_valores_a_sumar = [] # esta lista va a contener el valor que hay que sumar a cada año nuevo que se añade

for a in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_a_sumar.append((lista_datos_valor_df[a+1]-lista_datos_valor_df[a])/5)

lista_a_añadir_df = []# esta va a ser la lista final, con los valores a añadir por cada año a añadir

for valordf in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_años_nuevos = []# lista intermedia que guarda los valore por cada vuelta de grupo nuevo de años 
    lista_valores_años_nuevos.append(extrac_agri_df.iloc[valordf][2])
        
    for a in list(range(5)):   
        
        lista_valores_años_nuevos.append(lista_valores_años_nuevos[a]+lista_valores_a_sumar[valordf]) 
        valor_año_nuevo = lista_valores_años_nuevos[a]+lista_valores_a_sumar[a]

    lista_valores_años_nuevos.pop(0)
    lista_valores_años_nuevos.pop(len(lista_valores_años_nuevos)-1)
    lista_a_añadir_df.append(lista_valores_años_nuevos)

lista_a_añadir_df.pop(len(lista_a_añadir_df)-1)

    
    
 #CONVIERTO EL ARRAY DE DOS DIMENSIONES DE LOS AÑOS NUEVOS A AÑADIR-> lista_a_añadir_df EN UNA LISTA 
lista_nueva_valor = []
for indice in list(range(len(lista_a_añadir_df))):
    for subindice in list(range(len(lista_a_añadir_df[indice]))):
        lista_nueva_valor.append(lista_a_añadir_df[indice][subindice])
        
#AÑADO LA LISTA DE LOS AÑOS Y LA NUEVA LISTA OBTENIDA DEL ARRAY DE DOS DIMENSIONES EN EL DATAFRAM DF2_1


lista_años_nuevos = [1998,1999,2000,2001,2003,2004,2005,2006,2008,2009,2010,2011,2013,2014,2015,2016]

for indice in list(range(len(lista_nueva_valor))):
    df2_1.loc[len(df2_1)] =  ['Extracción de agua municipal', lista_años_nuevos[indice], lista_nueva_valor[indice]]  

In [ ]:
#ESTO YA FUNCION PARA EXTRACCION DE AGUA SUMA TODOS LOS SECTORES PARA TODOS LOS AÑOS

filtro = df2_1['Uso'].isin(['Extraccion total de agua (suma de sectores)'])
extrac_agri_df = df2_1[filtro]


# con esto extraigo los valores de los años, desde el principio hasta el final
lista_datos_valor_df = [] # esta lista va a contener los valores de la columna valor del dataframe
for a in list(range(len(extrac_agri_df))):
    lista_datos_valor_df.append(extrac_agri_df.iloc[a][2])
    

lista_valores_a_sumar = [] # esta lista va a contener el valor que hay que sumar a cada año nuevo que se añade

for a in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_a_sumar.append((lista_datos_valor_df[a+1]-lista_datos_valor_df[a])/5)

lista_a_añadir_df = []# esta va a ser la lista final, con los valores a añadir por cada año a añadir

for valordf in list(range(len(lista_datos_valor_df)-1)):
    lista_valores_años_nuevos = []# lista intermedia que guarda los valore por cada vuelta de grupo nuevo de años 
    lista_valores_años_nuevos.append(extrac_agri_df.iloc[valordf][2])
        
    for a in list(range(5)):   
            
        lista_valores_años_nuevos.append(lista_valores_años_nuevos[a]+lista_valores_a_sumar[valordf]) 
        valor_año_nuevo = lista_valores_años_nuevos[a]+lista_valores_a_sumar[a]

    lista_valores_años_nuevos.pop(0)
    lista_valores_años_nuevos.pop(len(lista_valores_años_nuevos)-1)
    lista_a_añadir_df.append(lista_valores_años_nuevos)

lista_a_añadir_df.pop(len(lista_a_añadir_df)-1)


#CONVIERTO EL ARRAY DE DOS DIMENSIONES DE LOS AÑOS NUEVOS A AÑADIR-> lista_a_añadir_df EN UNA LISTA 
lista_nueva_valor = []
for indice in list(range(len(lista_a_añadir_df))):
    for subindice in list(range(len(lista_a_añadir_df[indice]))):
        lista_nueva_valor.append(lista_a_añadir_df[indice][subindice])
        
#AÑADO LA LISTA DE LOS AÑOS Y LA NUEVA LISTA OBTENIDA DEL ARRAY DE DOS DIMENSIONES EN EL DATAFRAM DF2_1


lista_años_nuevos = [1998,1999,2000,2001,2003,2004,2005,2006,2008,2009,2010,2011,2013,2014,2015,2016]

for indice in list(range(len(lista_nueva_valor))):
    df2_1.loc[len(df2_1)] =  ['Extraccion total de agua (suma de sectores)', lista_años_nuevos[indice], lista_nueva_valor[indice]]


In [ ]:
#AHORA ELIMINO LOS AÑOS QUE NO ME INTERESAN

mask = df2_1[df2_1['Año'].isin([1997,1998,1999])].index
df2_1 = df2_1.drop(mask)

In [ ]:
df2_1.reset_index(inplace=True, drop=True)# despues de eliminar la primera fila, reseteo el index y dejamos la misma columna inicial del indice

In [ ]:
#ORDENO LOS VALORES POR AÑO Y USO

df2_1 =df2_1.sort_values(by=['Uso', 'Año'])

In [ ]:
#GUARDO EL DATAFRAM NORMALIZADO
df2_1.to_csv('Data/Extraccion de agua total por sector en Espana_normalizado.csv', sep= ';', index= False)

In [ ]:
# IMPORTO CSV DE FAO-> CULTIVOS Y DERIVADOS

df5 = pd.read_csv ('Data/Cultivos y derivados.csv', sep= ',',thousands= '.')
df5.info()

In [ ]:
# COMPRUEBO LOS VALORES NAN, SON PRODUCTOS QUE REALMNETE TIENEN VALOR 0
df5[df5['Valor'].isna()]

In [ ]:
# ELIMINO LOS NAN PORQUE SE TRATA DE PRODUCTOS QUE PARECE NO TENER PRODUCCCION
indice=df5[df5['Valor'].isna()].index

df5.drop(indice, inplace= True)

In [ ]:
#RESETEO EL INDICE

df5.reset_index(inplace=True, drop=True)
df5

In [ ]:
# AÑADO LA COLUMNA GRUPO PRODUCTOS
df5['Grupo productos'] = 'Cultivos y derivados'


In [ ]:
# IMPORTO CSV DE FAO-> GANADERIA PRIMARIA Y PROCESADA
df6 = pd.read_csv ('AGUA/FAOSTAT/Ganaderia primaria y procesada1.csv', sep= ',', thousands= '.')

In [ ]:
# ELIMINO LOS VALORES NAN DEL DATAFAME PORQUE NO SON RELEVANTES

indice=df6[df6['Valor'].isna()].index

df6.drop(indice, inplace= True)

In [ ]:
#AÑADO LA COLUMNA GRUPO PRODUCTOS
df6['Grupo productos'] = 'Ganadería primaria y procesada'

In [ ]:
#COMPRUEBO LOS VALORES QUE HAY EN UNIDADES PARA ELIMINAR LO QUE NO SEAN TONELADAS
df6['Unidad'].unique()

In [ ]:
#PROCEDO A ELIMINAR LO QUE NO SON TONELADAS
indice=df6[df6['Unidad'].isin(['1000 No.'])].index
indice

In [ ]:
# RESETEO EL INDICE Y COMPRUEBO
df6.reset_index (inplace=True, drop=True)

df6

In [ ]:
# HAGO UNA COPIA DEL DATAFRAME 5 PARA TRABAJAR SOBRE ESTE NUEVO

df7 = df5.copy()

In [ ]:
#INCORPORO EL DATAFRAME 6 EN EL 7 PARA PODER TRABAJAR CON LOS DATOS EN UN MISMO DATAFRAME

for indice in list(range(len(df6))):
    df7.loc[len(df7)] =  df6.loc[indice]


In [ ]:
#COMPRUEBO LOS AÑOS YA QUE SOLO QUIERO QUEDARME CON LA INFO HASTA EL 2018
df7['Año'].unique()

In [ ]:
# ELIMINO LOS AÑOS DEL 2019 AL 2020 Y COMPRUEBO

indice=df7[df7['Año'].isin([2019,2020])].index
indice

df7.drop(indice, inplace= True)

In [ ]:
#RESETEO EL INDICE DESPUES DE HABER ELIMINADO LOS AÑOS 2019 Y 2020
df7.reset_index (inplace=True, drop=True)

In [ ]:
# PROCEDO A GUARDARLO
df7.to_csv('Data/Cultivos y ganaderia_normalizado.csv', sep= ';', index= False)

In [ ]:
# PROCESO DE CREACION DE LOS GRAFICOS:

In [ ]:

import seaborn as sns
import matplotlib.pyplot as plt

import os
os.getcwd()

In [ ]:
#CREACION DE GRAFICOS SOBRE EL DATAFRAM DF1

df1 = pd.read_csv ('AGUA/EPDATA/evolucion_del_agua_disponible_en_españa_normalizado.csv', sep= ';' )
df1['Año'] = pd.to_datetime(df1['Año'])

In [ ]:
#GRAFICO DONDE SE MUESTRA LA EVOLUCION EN LOS AÑOS DEL AGUA TOTAL EN ESPAÑA

grafic_evo_agua_total = plt.figure(figsize=(12,7))
plt.style.use('dark_background')
plt.axes()# definimos los axes
plt.plot(df1["Año"],df1["Total Agua"], 'o-w', alpha = 0.7,label= 'Agua total', linewidth= 7, snap= True) # le decimos que pinte con plot, las lineas definidas dentro de la funcion con el valor de x y el de y
plt.ylim(80,115)
plt.xlabel ('Año', size= 15)
plt.ylabel ('10^9 m3/año', size= 15)
plt.title ('Evolución del agua disponible en España', fontdict= {'color': 'white', 'weight': 'bold', 'size': 18 }, loc= 'center', alpha = 0.7, pad=20)
plt.grid (False)

In [ ]:
#GRAFICO DONDE SE MUESTRA LA EVOLUCION EN LOS AÑOS DEL AGUA DISPONIBLE EN ESPAÑA
grafic_evo_tipos_agua = plt.figure(figsize=(12,7))
plt.style.use('dark_background')
plt.axes()# definimos los axes
plt.plot(df1["Año"],df1["No potabilizada Valor 10^9m3/año"], '--g', label= 'No potabilizada', linewidth= 4, snap= True) # le decimos que pinte con plot, las lineas definidas dentro de la funcion con el valor de x y el de y
plt.plot(df1["Año"],df1["Para su potabilización 10^9m3/año"], '--c', label= 'Para su potabilización', linewidth = 4, snap = True)# y aqui dentro es donde fijamos los valores de los ejes, de las etiquetas y del titulo

plt.xlabel ('Año', size= 15)
plt.ylabel ('10^9 m3/año', size= 15)
plt.title ('Evolución del agua disponible en España', fontdict= {'color': 'white', 'weight': 'bold', 'size': 18 }, loc= 'center', alpha=0.7, pad=20)
plt.grid (False)
plt.legend()

In [ ]:
# HAGO UN SET_INDEX Y A CONTINUACION UN DESCRIBE PARA COMPROBAR LOS MAXIMOS Y MINIMOS Y HACER LA COMPARACION ENTRE ELLOS Y PODER PRESENTARLA EN UN GRAFICO
df1_1 = df1.set_index('Año')
df1_1.describe()

In [ ]:
# ALMACENO EL MAXIMO Y MINMO EN DOS VARIABLES PARA GRAFICARLOS

data_min = df1[['No potabilizada Valor 10^9m3/año','Para su potabilización 10^9m3/año']].min()
data_max = df1[['No potabilizada Valor 10^9m3/año','Para su potabilización 10^9m3/año']].max()

In [ ]:
# GENERO UN SUBPLOT Y ASI LOS MUESTRO EN LA MISMA IMAGEN

graf_comp_max_min=plt.figure(figsize=(15,7))
plt.style.use('dark_background')
plt.subplot(1,2,1)


colores = ('#9ACD32', '#87CEEB')

plt.pie(data_min.values,
        labels=data_min.index,
        autopct='%1.2f%%',
        colors= colores, shadow = True, startangle= 20, textprops= {'fontsize':15})
plt.title('Máximo. Año 2004',fontsize = 15 )

plt.subplot(1,2,2)

plt.pie(data_max.values,
        labels=data_max.index,
        autopct='%1.2f%%',
        colors= colores, shadow = True, startangle= 20,textprops= {'fontsize':15})
plt.title('Mínimo. Año 2018',fontsize = 15)

plt.show();


In [ ]:
# AHORA SACAMOS LA DIFERENCIA DEL TOTAL DE AGUA DISPONIBLE A NIVEL PORCENTUAL ENTRE EL EL MAXIMO Y EL MÍNIMO PARA PODER VER CUANTO A DISMINUIDO
# TAMBIEN SACAMOS LA DIFERENCIA DEL AGUA POTABLE A NIVEL PORCENTUAL ENTRE EL MAXIMO Y EL MÍNIMO PARA PODER COMPARAR ESTE PORCENTAJE Y EL OTRO
#Y DE ESTA MANERA VER SI ESTAMOS CONSUMIENDO MAS AGUA AUNQUE HAYA DECRECIDO


#       No potabilizada Valor 10^9m3/año	Para su potabilización 10^9m3/año	               %SOBRE EL DE LAS DIFERENCIAS        Total Agua


# min	38.757420	                        47.697090	                                         55,170152%                          86.454510

#DIF                                       -13.16856                                            -54,97322 %                          23,9545

#DIF %%!                               UN  21,63545%  MENOS DE AGUA PARA SU POTABILIZACION                                          UN 21,69615% MENOS DE AGUA TOTAL

# max	49.543360	                        60.865650	                                         55,12743% %                         110.409010

In [ ]:
#GUARDO LOS TRES GRAFICOS 

grafic_evo_agua_total.savefig('Graficos/Grafico_agua_total_disponible.png')
grafic_evo_tipos_agua.savefig('Graficos/Grafico_evo_tipos_agua.png')
graf_comp_max_min.savefig('Graficos/Grafico_comp_max_min.png')

In [ ]:
#CREACION DE GRAFICOS SOBRE EL DATAFRAM DF2

df2 = pd.read_csv ('Data/Extraccion de agua total por sector en Espana_normalizado.csv', sep= ';', index_col= False)

In [ ]:
# EXTRAIGO LOS DATOS DE CADA SECTOR EN UN DATAFRAME Y LOS GRAFICO PARA PODER VER LA COMPARACION DE LOS DIFERENTES CONSUMOS

df2_agricultura = df2[df2['Uso']=='Extracción de agua agrícola']
df2_industrial = df2[df2['Uso']=='Extracción de agua industrial']
df2_municipal = df2[df2['Uso']=='Extracción de agua municipal']

In [ ]:
# UTILIZO UN PLOT PARA GRAFICAR LOS TRES. 


grafic_consu_por_tipos = plt.figure(figsize=(12,7))
plt.style.use('dark_background')
plt.axes()# definimos los axes
plt.plot(df2_agricultura["Año"],df2_agricultura["Valor 10^9m3/año"], 'r-X', label= 'Agrícola', linewidth= 2, snap= True ) # le decimos que pinte con plot, las lineas definidas dentro de la funcion con el valor de x y el de y
plt.plot(df2_industrial["Año"],df2_industrial["Valor 10^9m3/año"], '-*', label= 'Industrial', linewidth = 2, snap = True)# y aqui dentro es donde fijamos los valores de los ejes, de las etiquetas y del titulo
plt.plot(df2_municipal["Año"],df2_municipal["Valor 10^9m3/año"], '->', label= 'Municipal', linewidth = 2, snap = True) # y aqui dentro es donde fijamos los valores de los ejes, de las etiquetas y del titulo
# plt.plot(df1["Año"],df1["Total Agua"], 'o-w', alpha = 0.7,label= 'Agua total', linewidth= 7, snap= True)
plt.xlabel ('Año', size= 15)

plt.ylabel ('10^9 m3/año', size= 15)
plt.title ('Consumo por sector', fontdict= {'color': 'white', 'weight': 'bold', 'size': 16 }, loc= 'center', pad=20, alpha= 0.)
plt.grid (False, alpha=0.1)
plt.legend()

In [ ]:
# AHORA VOY A GRAFICAR LOS MISMOS DATOS CON UN DIAGRAMA DE BARRAS APILADAS PARA QUE SE VEA CON CLARIDAD EL PESO DE CADA SECTOR

anos = list(range(len(df2_agricultura['Año'])))
indice= np.arange(len(anos))

grafi_barras_acum_por_tipos_agua= plt.figure(figsize=(15,7))
ancho_barras = 0.50


plt.bar(indice, df2_agricultura['Valor 10^9m3/año'], label= 'Agricultura', width= ancho_barras,color= 'DarkOliveGreen', edgecolor= 'Gray')
plt.bar(indice, df2_industrial['Valor 10^9m3/año'], width=ancho_barras, label='industrial', bottom= df2_agricultura['Valor 10^9m3/año'],color= 'Goldenrod', edgecolor ='Gray')
plt.bar(indice, df2_municipal['Valor 10^9m3/año'], width=ancho_barras, label='municipal',bottom= np.array(df2_agricultura['Valor 10^9m3/año'])+np.array(df2_industrial['Valor 10^9m3/año']), color = 'DarkKhaki', edgecolor = 'Gray')
plt.xticks(indice,df2_agricultura['Año'])
plt.ylabel ('10^9 m3/año', size= 15)
plt.title ('Consumo por sector', fontdict= {'color': 'white', 'weight': 'bold', 'size': 16 }, loc= 'center', pad=20, alpha= 0.)
plt.grid (True, alpha=0.1)

plt.legend()

plt.show()

In [ ]:
# QUIERO MOSTRAR EL PESO DEL CONSUMO AGRICULA DEL 2018 PARA QUE SE PUEDA VER CON MUCHA MAS CLARIDAD EL PORCENTAJE QUE SUPONE ASI QUE GUARDO LA INFO EN UNA VARIABLE

data_2018 = df2[df2['Año']== 2018]
data_2018.drop(data_2018.index[[0]], axis=0, inplace=True)

In [ ]:
# UTILIZO UN PIE PARA QUE SE REPRESENTE CON CLARIDAD

colores = ('DarkOliveGreen', 'Goldenrod', 'DarkKhaki')
grafi_2018_cons_agri_total= plt.figure(figsize=(9,9))

plt.pie(data_2018['Valor 10^9m3/año'],
        autopct='%1.2f%%',
        colors= colores, shadow = True, startangle= 20, textprops= {'fontsize':15})
plt.title('Año 2018',fontsize = 15, loc= 'center')
plt.legend(data_2018['Uso'],loc='upper right')

plt.show();

In [ ]:
# GUARDO LOS TRES GRAFICOS

grafi_2018_cons_agri_total.savefig('grafico_2018_cons_agri_total.png')
grafic_consu_por_tipos.savefig('Grafico_consu_por_tipos.png')
grafi_barras_acum_por_tipos_agua.savefig('Grafico_barras_acum_por_tipos_Agua.png')

In [ ]:
# CREACION DE GRAFICOS DF7

df7 = pd.read_csv ('Data/Cultivos y ganaderia_normalizado.csv', sep= ';', index_col= False, thousands= '.')

In [ ]:
# TENGO QUE SACAR LOS SUMATORIOS TOTALES POR AÑO Y POR GRUPO DE PRODUCTOS PARA PODER HACER LA GRAFICA DE LA PRODUCCION DE CADA SECTOR

df7_copy = df7.copy()
df7_copy = df7_copy[['Grupo productos','Año','Valor']]

df7_copy_agrupado = df7_copy.groupby(['Grupo productos','Año']).sum()

df7_copy_agrupado

In [ ]:
#TENGO QUE CREAR UN DATAFRAME NUEVO PARA PODER HACER LA GRAFICA SOBRE ESTE
diccionario = {'Grupo productos': [], 'Año': [], 'Valor': []}
lista_años_nuevos = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]
df7_sumatotalano_grupoproducto = pd.DataFrame(diccionario)
df7_sumatotalano_grupoproducto

In [ ]:
# VOY RELLENANDO LAS FILAS CON LA INFORMACION DEL DATA AGRUPADO PARA LA PARTE DE CULTIVOS Y DERIVADOS
for indice in list(range(19)): # es decir por el valor del indice sobre el total de la longitud (38)
                  df7_sumatotalano_grupoproducto.loc[len(df7_sumatotalano_grupoproducto)] = 'Cultivos y derivados' , lista_años_nuevos[indice], df7_copy_agrupado.iloc[indice][0]
df7_sumatotalano_grupoproducto

In [ ]:
# VOY RELLENANDO LAS FILAS CON LA INFORMACION DEL DATA AGRUPADO PARA LA PARTE DE PRODUCTOS DE GANADERIA
for indice in list(range(19,20+18)): # es decir por el valor del indice sobre el total de la longitud (38)
                  df7_sumatotalano_grupoproducto.loc[len(df7_sumatotalano_grupoproducto)] = 'Ganadería primaria y procesada', lista_años_nuevos[indice-19], df7_copy_agrupado.iloc[indice][0]
df7_sumatotalano_grupoproducto

In [ ]:
# COMPRUEBO
df7_sumatotalano_grupoproducto

In [ ]:
# DIVIDO LAS TONELADAS ENTRE 10^6
df7_sumatotalano_grupoproducto['Valor'] = df7_sumatotalano_grupoproducto['Valor']/10**6
df7_sumatotalano_grupoproducto

In [ ]:
# SACO EL GRAFICO DE LA EVOLUCION DE LOS ALIMENTOS DURANTE LOS AÑOS PARA QUE SE PUEDA APRECIA QUE REALMENTE DENTRO DEL SECTOR DE LA AGRICULTURA, ESTA INCLUIDO LA 
#PRODUCCION DE PRODUCTOS GANDEROS

graf_evo_grupo_produ = plt.figure(figsize=(20,9)) 

sns.barplot(x = 'Año', y = 'Valor', data = df7_sumatotalano_grupoproducto, hue = "Grupo productos", ci= False)

plt.ylabel ('10^6 Toneladas/año', size= 13)


In [ ]:
# GUARDO EL GRAFICO  

graf_evo_grupo_produ.savefig('grafico_barras_evo_grupo_produ.png')

In [ ]:
# ESTOY EXTRAYENDO EL PORCENTAJE DE CRECIMIENTO DE LA PRODUCCIONA DE GANADERIA PRIMARIA Y PROCESADA QUE SE INCLUYE DENTRO DEL SECTOR DE LA AGRICULTURA,
# PARA VER QUE EL PORCENTAJE ES POSITIVO, EXPLICAR QUE PESE A QUE EL AGUA DISMINIUYE, ESTA PRODUCCION HA AUMENTADO
gana_prim_proce = df7[df7['Grupo productos']== 'Ganadería primaria y procesada']
gana_prim_proce_2000 = gana_prim_proce[gana_prim_proce['Año']==2000]['Valor'].sum()
gana_prim_proce_2018 = gana_prim_proce[gana_prim_proce['Año']==2018]['Valor'].sum()
incremento = round(((gana_prim_proce_2018/gana_prim_proce_2000)*100-100),2)
incremento

In [ ]:
# PARA EL GRAFICO DE LAS PALABRAS UTILICE TABLEAU E HICE UN REFCORTE DE LA PANTALLA